<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Application/NLP/CFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk

grammar1 = nltk.CFG.fromstring("""
  S -> QP | VP SP | QP SP | O SP | O VBP | WP | SP
  SP -> PRP NN | PRP IN | NN
  QP -> WP | WRB A2
  O -> PRP | PRP A1 | PRP A2 A3 | PRP
  A1 -> "is"
  A2 -> "are"
  A3 -> "ay"
  IN -> "from"
  WP -> "What" | "How" | "When"
  WDT -> "Which"
  WRB -> "Where"
  NN -> "name" | "estudyante"
  PRP -> "you" | "I"
  RB -> "here" 
  VBP -> "live"
  """)
# (S (QP (WP What)) (SP (PRP you) (NN name)))
grammar2 = nltk.CFG.fromstring("""
  S -> QP | VP SP | QP SP | O SP | O VBP
  SP -> "NN" | "NN PRP" 
  QP -> "WP" | "WP A1"
  O -> "PRP" | "PRP A3" | "PRP A2 A3"
  """)

grammar4 = nltk.CFG.fromstring("""
  S -> A
  A -> QP SP | 
  SP -> NN | PRP NN 
  QP -> WP 
  O -> A1 PRP | PRP A3 | PRP A2 A3
  WP -> "What"
  PRP -> "you"
  NN -> "name"
  A1 -> "is"
  """)
 	
grammar3 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

# What's your name (What name you?) - S -> QP SP -> QP -> WP A1 -> "Ano" -> "ang" -> SP -> NN PRP -> NN -> "pangalan" -> PRP -> "ikaw"
# Are you a student? (you student?) - S -> O SP -> O -> PRP A2 A3 -> PRP -> "ikaw" -> A2 -> "ba" -> A3 -> "ay" -> SP -> NN -> "estudyante"
# I'm good (I good) - S -> O VBP -> O -> PRP A2 -> PRP -> "ako" -> A2 -> "ay" -> VBP -> "mabuti"
# Where you from 
# I live here

In [ ]:
grammar1.productions()

[S -> QP,
 S -> VP SP,
 S -> QP SP,
 S -> O SP,
 S -> O VBP,
 S -> WP,
 S -> SP,
 SP -> PRP NN,
 SP -> PRP IN,
 SP -> NN,
 QP -> WP A1,
 QP -> WRB A2,
 O -> PRP,
 O -> PRP A1,
 O -> PRP A2 A3,
 O -> PRP,
 A1 -> 'is',
 A2 -> 'are',
 A3 -> 'ay',
 IN -> 'from',
 WP -> 'What',
 WP -> 'How',
 WP -> 'When',
 WDT -> 'Which',
 WRB -> 'Where',
 NN -> 'name',
 NN -> 'estudyante',
 PRP -> 'you',
 PRP -> 'I',
 VBP -> 'live']

In [ ]:
import random


d = {}
 
d["b a c"]=[["a", "b"]]
d["a"]=[["what", "e"],["when", "e"]]
d["b"]=[["c", "d"]]
d["c"]=[["you"]]
d["d"]=[["name"],["bday"]]
d["e"]=[["is"]]

expansion = []
seed = "b a c"


def expand(start, expansion):
    if start in d:
    #Grab one possible expansion
        possible = d[start]
        random_expansion = random.choices(possible)
    # Call this method again with the current element
        for i in range(len(random_expansion)):
            expand(random_expansion[i], expansion);
 
  # if the rule wasn't found, then it's a terminal:
  # append the string to the expansion
    else:
        expansion.append(start)

expand(seed, expansion)

print(expansion)

TypeError: ignored

In [ ]:

text = "What you name"
sent = text.split()
# rd_parser = nltk.parse.shiftreduce.ShiftReduceParser(grammar1)
rd_parser = nltk.parse.recursivedescent.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
  print(tree)
# nltk.parse.shiftreduce.demo()
# nltk.parse.recursivedescent.demo()

In [ ]:
from textblob import TextBlob

text = "Get ball"

blob = TextBlob(text)
blob.tags           # [('The', 'DT'), ('titular', 'JJ'),
                    #  ('threat', 'NN'), ('of', 'IN'), ...]

[('Get', 'VB'), ('the', 'DT'), ('ball', 'NN')]

In [ ]:
nltk.download('treebank')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True